In [1]:
import networkx as nx
from collections import deque

In [2]:
G = nx.read_weighted_edgelist("ff2.edgelist", create_using=nx.DiGraph(), nodetype=int)

### (1)

In [3]:
def find_augmentpath(N, s, t):

    # πの初期化
    Pi = [-1] * nx.number_of_nodes(N)

    # 訪れた点を格納するvisitedの初期化
    visited = set()

    # stackの初期化
    stack = deque()

    # スタックに最初に訪れるsを入れる
    stack.appendleft(s)

    # stackの中身が存在する間処理を行う。
    while stack:

        # 次に訪問する点をstackの先頭から取り出す
        v = stack.popleft()

        # 次の点がtのとき
        if v == t:

            # πとtrueのタプルを返す
            return (Pi, True)

        # vが未訪問のとき
        if v not in visited:

            # vを訪問済みにする
            visited.add(v)

            # vの隣接点を一つずつ取り出してwに代入
            for w in N.neighbors(v):

                # wが未訪問で辺v,wの重さが0でないとき
                if w not in visited and N.edges[v, w]['weight'] != 0:

                    # stackにwを入れる
                    stack.appendleft(w)

                    # wはvから訪れたことを記録
                    Pi[w] = v

    # tまでたどり着けなかったら、πとfalseを返す
    return (Pi, False)

In [4]:
print(find_augmentpath(G, 0, 5))

([-1, 2, 0, 4, 2, 4], True)


### (2)Ford-Fulkerson

In [5]:
def restore_shortestpath(u, v, Pi):
    path = []
    temp = v
    while temp != u:
        parent = Pi[temp]
        path.append((parent, temp))
        temp = parent
    path.reverse()
    return path

In [6]:
def min_capacity(N, path):
    min_cap = float("inf")
    for u, v in path:
        capacity = N.edges[u, v]["weight"]
        if capacity < min_cap:
            min_cap = capacity
    return min_cap

In [7]:
def increase_flow(G, N, path, amount, flow):
    for u, v in path:
        if G.has_edge(u, v):
            flow[(u, v)] += amount
        else:
            flow[(v, u)] -= amount

        N.edges[u, v]["weight"] -= amount
        
        if not N.has_edge(v, u):
            N.add_edge(v, u, weight = 0)

        N.edges[v, u]["weight"] += amount

In [8]:
def my_Ford_Fulkerson(G:nx.classes.DiGraph, s:int, t:int):
    N = G.copy()
    f= {}

    for u , v in N.edges:
        f[(u, v)] = 0
    Pi, is_found = find_augmentpath(N, s, t)

    while is_found:

        augmentpath = restore_shortestpath(s, t, Pi)
        min_cap = min_capacity(N, augmentpath)
        
        increase_flow(G, N, augmentpath, min_cap, f)
        Pi, is_found = find_augmentpath(N, s, t)


    return N, f

In [12]:
N, f = my_Ford_Fulkerson(G, 0, 5)
print( [[(s, t) ,N.edges[s, t]["weight"]] for s, t in N.edges()])
print(f)

[[(0, 1), 0.0], [(0, 2), 0.0], [(1, 3), 0.0], [(1, 2), 2.0], [(1, 0), 10.0], [(2, 1), 2.0], [(2, 4), 3.0], [(2, 0), 13.0], [(3, 2), 9.0], [(3, 5), 1.0], [(3, 4), 7.0], [(3, 1), 12.0], [(4, 3), 0.0], [(4, 5), 0.0], [(4, 2), 11.0], [(5, 4), 4.0], [(5, 3), 19.0]]
{(0, 1): 10.0, (0, 2): 13.0, (1, 3): 12.0, (2, 1): 2.0, (2, 4): 11.0, (3, 2): 0, (3, 5): 19.0, (4, 3): 7.0, (4, 5): 4.0}


In [ ]:
def increase_flow(G, N, path, amount, flow):
    # pathからu,vを取り出す
    for u, v in path:
        # Gに辺u,vが存在するなら
        if G.has_edge(u, v):
            # u,vのフローを増やす
            flow[(u, v)] += amount
        else:
            # v,uのフローを減らす
            flow[(v, u)] -= amount

        # Nの辺u,vの重みを減らす
        N.edges[u, v]["weight"] -= amount
        
        # Nに辺v,uがないとき重み0で辺v,uを作成
        if not N.has_edge(v, u):
            N.add_edge(v, u, weight = 0)

        # Nの辺v,uの重みを増やす
        N.edges[v, u]["weight"] += amount

In [ ]:
def my_Ford_Fulkerson(G:nx.classes.DiGraph, s:int, t:int):
    # GのコピーNを作る
    N = G.copy()
    # フローを空集合で作る
    f= {}
    # Nの辺u,vすべてを0で初期化
    for u , v in N.edges:
        f[(u, v)] = 0
    # 道のりπと道が存在するかをそれぞれ、piとis_foundに代入
    Pi, is_found = find_augmentpath(N, s, t)

    # 道が存在する間処理を行う。
    while is_found:
        # sからtまでの道のりを取得
        augmentpath = restore_shortestpath(s, t, Pi)
        # sからtまでの道のりの中で最小の値を取得
        min_cap = min_capacity(N, augmentpath)
        # augmentpathの道のりの重みやフローを更新
        increase_flow(G, N, augmentpath, min_cap, f)
        # 道のりπと道が存在するかをそれぞれ、piとis_foundに代入
        Pi, is_found = find_augmentpath(N, s, t)

    # 最終的な残余ネットワークNと最大フローfを返す
    return N, f